In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import glob

In [4]:
# Define the file paths for the SSD and TD dataset labels
uxtd_path = '/content/drive/My Drive/Colab Notebooks/SummerProject/data/labels-uxtd-uxssd-upx/uxtd/word_labels/lab/'
uxssd_path = '/content/drive/My Drive/Colab Notebooks/SummerProject/data/labels-uxtd-uxssd-upx/uxssd/word_labels/lab/'

#Use glob to create a list of individual file paths
all_files = glob.glob(os.path.join(uxtd_path, "*.lab"))
print(len(all_files))
all_files = all_files + glob.glob(os.path.join(uxssd_path, "*.lab"))
print(len(all_files))

1567
5067


In [19]:
all_files[5050][103:-4]

'01M-Maint2-005A'

In [16]:
# Explore how best to slice the patient name and sub group

all_files[5050][103:-4]

patient = all_files[5050][103:106]
print("Patient " + patient)

sub_group = all_files[5050][107:-9]
print("Sub group "+ sub_group)


sess = all_files[5050][-8:-4]
print("Session " + sess)

print("/content/drive/My Drive/Colab Notebooks/SummerProject/data/core-uxssd/core/"+patient+"/"+sub_group+"/"+sess+".wav")

Patient 01M
Sub group Maint2
Session 005A
/content/drive/My Drive/Colab Notebooks/SummerProject/data/core-uxssd/core/01M/Maint2/005A.wav


In [ ]:
# Iteratively extract each label file, this includes two timings and the relative word separated by a space
# Then slice the file name to append SSD diagnosis, Patient Name and Therapy session to the dataframe

temp_li = []
x=0

for filename in all_files:
  x=x+1
  print(x)
  temp_df = pd.read_csv(filename, sep=' ', index_col=None, header=0)
  td_ssd = "uxtd" if "/uxtd/" in filename else "uxssd"
  if td_ssd == "uxtd":
    sess = filename[-8:-4]
    patient = filename[-12:-9]
    temp_df['td_ssd'] = "td"
    temp_df['filepath'] = "/content/drive/My Drive/Colab Notebooks/SummerProject/data/core-uxtd/core/"+patient+"/"+sess+".wav"
  else:
    patient = filename[103:106]
    sub_group = filename[107:-9]
    sess = filename[-8:-4]
    temp_df['td_ssd'] = "ssd"
    temp_df['filepath'] = "/content/drive/My Drive/Colab Notebooks/SummerProject/data/core-uxssd/core/"+patient+"/"+sub_group+"/"+sess+".wav"

  #convert dataframe to list for expansion
  temp_li.append(temp_df.values.tolist())

In [21]:
# Expand list and convert to dataframe with new column headers
li = [x for xs in temp_li for x in xs]
df = pd.DataFrame(li, columns = ['Start', 'End', 'word', 'td_ssd', 'filepath'])

# Convert number format from milliseconds to seconds
df['End'] = df['End'] * (1*(10**-7))
df['End'] = df['End'].round(decimals=2)
df['Start'] = df['Start'] * (1*(10**-7))
df['Start'] = df['Start'].round(decimals=2)


In [23]:
# Slice dataframe by only the words we want
common = ['SWING', 'UMBRELLA', 'QUEEN', 'HELICOPTER', 'ORANGE']
df = df.loc[df['word'].isin(common)]
df

In [ ]:
# Save the dataframe to CSV to save re-doing this processing every time we train
df.to_csv('/content/drive/My Drive/Colab Notebooks/SummerProject/data/siamese_1_GPU_DF.csv', index=True)